<a href="https://colab.research.google.com/github/grt1999/7VFI.AI/blob/main/7ViFi_Transformer_by_JLF_by_Lisa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch

In [ ]:
import torch

In [ ]:
try:
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)

device = (
    "GPU" # Mac mps
    if getattr(torch, "has_mps", False)
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Note: using Google CoLab
Using device: cuda


<ipython-input-3-fa34dde8fd97>:12: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  if getattr(torch, "has_mps", False)


In [ ]:
# import all necceary libraries
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import requests
import io
import datetime as dt

In [ ]:
#import data from github link
url = "https://raw.githubusercontent.com/BACCHUS2333/homework2.github.io/main/MergedFinalData3.0.csv"
s=requests.get(url).content
all_data=pd.read_csv(io.StringIO(s.decode('utf-8'))).iloc[:,1:]

all_data['TVL'] = all_data['TVL'].astype(float)
all_data['date'] = pd.to_datetime(all_data['date'])
all_data['year'] = all_data['date'].dt.year

print("data updated (time: {})".format(dt.datetime.now()+dt.timedelta(minutes = 0)))






data updated (time: 2023-12-15 08:27:55.042606)


In [ ]:
# overview of data frame
all_data.head()

,protocol,date,TVL,Fee,Supply,Staking,Unique Addresses,User Growth,price,amount,...,New Addresses,Active Addresses,Zero Addresses,Number of Transactions,volume,Unnamed: 3,code quality and transparency,biggest drawdown,Correlation,year
0,dydx,2021-09-10,4.365587e+08,0,2.808704e+06,0,87.888087,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.292133e+09,NaN,9.0,7.401246e+08,2,2021
1,dydx,2021-09-10,4.848482e+08,0,2.593049e+05,0,87.888087,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.292133e+09,NaN,9.0,7.401246e+08,2,2021
2,dydx,2021-09-11,4.476166e+08,0,1.935136e+06,0,87.888087,0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.632538e+08,NaN,9.0,7.401246e+08,2,2021
3,dydx,2021-09-11,4.848482e+08,0,2.593049e+05,0,87.888087,0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.632538e+08,NaN,9.0,7.401246e+08,2,2021
4,dydx,2021-09-12,4.533061e+08,0,1.231117e+06,0,87.888087,0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.025489e+08,NaN,9.0,7.401246e+08,2,2021


In [ ]:
# Model definition using Transformer

class TransformerModel(nn.Module):
   def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
          super(TransformerModel, self).__init__()

          self.encoder = nn.Linear(input_dim, d_model)
          self.pos_encoder = PositionalEncoding(d_model, dropout)
          encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
          self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
          self.decoder = nn.Linear(d_model, 1)

   def forward(self, x):
          x = self.encoder(x)
          x = self.pos_encoder(x)
          x = self.transformer_encoder(x)
          x = self.decoder(x[:, -1, :])
          return x


class PositionalEncoding(nn.Module):
      def __init__(self, d_model, dropout=0.1, max_len=5000):
          super(PositionalEncoding, self).__init__()
          self.dropout = nn.Dropout(p=dropout)

          pe = torch.zeros(max_len, d_model)
          position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
          div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
          pe[:, 0::2] = torch.sin(position * div_term)
          pe[:, 1::2] = torch.cos(position * div_term)
          pe = pe.unsqueeze(0).transpose(0, 1)
          self.register_buffer('pe', pe)

      def forward(self, x):
          x = x + self.pe[:x.size(0), :]
          return self.dropout(x)


  # train test split
def to_sequences(seq_size, obs):
      x = []
      y = []
      for i in range(len(obs) - seq_size):
          window = obs[i:(i + seq_size)]
          after_window = obs[i + seq_size]
          x.append(window)
          y.append(after_window)
      return torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), torch.tensor(y, dtype=torch.float32).view(-1, 1)


#FINAL FUNCTION
#---------------------------------------------
#---------------------------------------------
def transformer(inputdata,protocol,var='TVL'):
    df = inputdata


#CORRECTION FIRST:
#************************************
#************************************
#some of the protocols do not have data before 2022 so I changed the way we select the dataset
    #split the train and test
    df_train = df[:len(df)*4//5]
    df_test = df[len(df)*4//5:]
    #df_train = df[df['year'] < 2022]
    #df_test = df[df['year'] >= 2022]
#************************************
#************************************


    protocol_train = inputdata[var].to_numpy().reshape(-1, 1)
    protocol_test = inputdata[var].to_numpy().reshape(-1, 1)


    scaler = StandardScaler()

#CORRECTION SECOND:
#************************************
#************************************
    #protocol_train = scaler.fit_transform(protocol_train).flatten().tolist()
    #protocol_test = scaler.transform(protocol_test).flatten().tolist()
    protocol_train = scaler.fit_transform(protocol_train).ravel().tolist()
    protocol_test = scaler.transform(protocol_test).ravel().tolist()
#************************************
#************************************


  # Sequence Data Preparation
    SEQUENCE_SIZE = 10


    x_train, y_train = to_sequences(SEQUENCE_SIZE, protocol_train)
    x_test, y_test = to_sequences(SEQUENCE_SIZE, protocol_test)


  # Setup data loaders for batch
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(x_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


  # Positional Encoding for Transformer




    model = TransformerModel().to(device)

  # Train the model
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

    epochs = 1000
    early_stop_count = 0
    min_val_loss = float('inf')

    for epoch in range(epochs):
      model.train()
      for batch in train_loader:
          x_batch, y_batch = batch
          x_batch, y_batch = x_batch.to(device), y_batch.to(device)

          optimizer.zero_grad()
          outputs = model(x_batch)
          loss = criterion(outputs, y_batch)
          loss.backward()
          optimizer.step()
      model.eval()
      val_losses = []
      with torch.no_grad():
          for batch in test_loader:
              x_batch, y_batch = batch
              x_batch, y_batch = x_batch.to(device), y_batch.to(device)
              outputs = model(x_batch)
              loss = criterion(outputs, y_batch)
              val_losses.append(loss.item())

      val_loss = np.mean(val_losses)
      scheduler.step(val_loss)

      if val_loss < min_val_loss:
          min_val_loss = val_loss
          early_stop_count = 0
      else:
          early_stop_count += 1

      if early_stop_count >= 5:
          print("Early stopping!")
          break
      print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")


    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch = x_batch.to(device)
            outputs = model(x_batch)

#CORRECTION THREE:
#************************************
#************************************
            #predictions.extend(outputs.squeeze().tolist())
            #predictions.extend(outputs.squeeze().cpu().numpy().tolist())
            predictions.extend(outputs.cpu().numpy().flatten().tolist())
#************************************
#************************************


    rmse = np.sqrt(np.mean((scaler.inverse_transform(np.array(predictions).reshape(-1, 1)) - scaler.inverse_transform(y_test.numpy().reshape(-1, 1)))**2))
    print(f"Score (RMSE): {rmse:.4f}")
    print(predictions)
    final_result_df.loc[len(final_result_df)] = [protocol,rmse,predictions]
    return final_result_df




start transformer engine:


In [ ]:
#RUN THE TRANSFORMER FUNCTION on each protocol respectively
print("start transformer engine: {}".format(dt.datetime.now()))
for protocols in all_data['protocol'].unique():
    transformer(all_data[all_data['protocol'] == protocols],protocols)

start transformer engine: 2023-12-15 08:28:04.222961


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/1000, Validation Loss: 1.0020
Epoch 2/1000, Validation Loss: 0.9934
Epoch 3/1000, Validation Loss: 0.9881
Epoch 4/1000, Validation Loss: 1.0322
Epoch 5/1000, Validation Loss: 1.0398
Epoch 6/1000, Validation Loss: 0.9341
Epoch 7/1000, Validation Loss: 0.9491
Epoch 8/1000, Validation Loss: 1.0033
Epoch 9/1000, Validation Loss: 0.9676
Epoch 00010: reducing learning rate of group 0 to 5.0000e-04.
Epoch 10/1000, Validation Loss: 0.9683
Early stopping!
Score (RMSE): 196580665.3250
[0.27272486686706543, 0.25255727767944336, 0.24924223124980927, 0.21416623890399933, 0.21878184378147125, 0.13821004331111908, 0.21931396424770355, 0.135795459151268, 0.21737851202487946, 0.15211980044841766, 0.21770717203617096, 0.16313308477401733, 0.22782357037067413, 0.1397596150636673, 0.23289592564105988, 0.09980183839797974, 0.24022959172725677, 0.07270370423793793, 0.24730397760868073, 0.01481713354587555, 0.22694535553455353, -0.06771489232778549, 0.2402612417936325, -0.061447806656360626, 0.271094

NameError: ignored

In [ ]:
final_result = final_result_df[['Protocol','rmse']]
final_result

,Protocol,rmse
0,dydx,1.955723e+08
1,umami,2.184247e+08
2,gns,1.796249e+08
3,bifi,2.180774e+08
4,looks,2.186220e+08
5,rbn,2.110899e+08
6,rdt,4.906259e+07
7,dpx,2.016050e+08
8,gmx,2.040404e+08
9,crv,5.349974e+07


In [ ]:
def format_number(number):
    num = float(number)
    scaled_num = num / 1e8
    return f"{scaled_num:.0f}"

# Apply the function to the DataFrame column
final_result['rmse'] = final_result['rmse'].apply(format_number)
final_result

<ipython-input-11-8aff62b6d386>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result['rmse'] = final_result['rmse'].apply(format_number)


,Protocol,rmse
0,dydx,2
1,umami,2
2,gns,2
3,bifi,2
4,looks,2
5,rbn,2
6,rdt,0
7,dpx,2
8,gmx,2
9,crv,1


In [ ]:
# Define a function to identify risk
def get_rank_select(num):
    try:
        num = int(num)  # Convert num to an integer
    except ValueError:
        return "Invalid value"  # Return this if num cannot be converted to an integer

    if 0 <= num <= 2:
        return 'Low'
    elif 3 <= num <= 5:
        return 'Medium'
    elif 6 <= num <= 8:
        return 'Medium High'
    elif 9 <= num <= 10:
        return 'High'
    else:
        return "Invalid value"

# Apply the function to the DataFrame column
final_result['Risk Level'] = final_result['rmse'].apply(get_rank_select)
final_result


,Protocol,rmse,Risk Level
0,dydx,2,Low
1,umami,2,Low
2,gns,2,Low
3,bifi,2,Low
4,looks,2,Low
5,rbn,2,Low
6,rdt,0,Low
7,dpx,2,Low
8,gmx,2,Low
9,crv,1,Low


In [ ]:
# Define a dictionary with the old values as keys and the new values as values
rename_mapping = {
    'dydx': 'DYDX',
    'umami': 'Umami',
    'gns': 'Gains Network',
    'bifi': 'Beefy',
    'looks': 'LooksRare',
    'rbn': 'Ribbon Finance',
    'rdt': 'Redacted Finance',
    'dpx': 'Dopex',
    'gmx': 'GMX',
    'crv': 'Curve',
    'snx': 'Synthetix'
}

# Use the `replace` method to rename the values
final_result['Protocol'] = final_result['Protocol'].replace(rename_mapping)
final_result

# renmae rmse column
final_result = final_result.rename(columns={'rmse': 'Prediction Output'})
final_result

,Protocol,Prediction Output,Risk Level
0,DYDX,2,Low
1,Umami,2,Low
2,Gains Network,2,Low
3,Beefy,2,Low
4,LooksRare,2,Low
5,Ribbon Finance,2,Low
6,Redacted Finance,0,Low
7,Dopex,2,Low
8,GMX,2,Low
9,Curve,1,Low


In [ ]:
import matplotlib.pyplot as plt
plt.bar(final_result_df['Protocol'],final_result_df['rmse'])
plt.xlabel("protocols name")
plt.ylabel("rmse")
plt.title("transformer result: rmse of protocols")rank

## Adding the latest metrics of each protocols to the final_result data frame for presentation and as the fruit of our project

In [ ]:
#acess to the data
url = 'https://raw.githubusercontent.com/BACCHUS2333/homework2.github.io/main/df_for_pre.csv'
s=requests.get(url).content
all_data=pd.read_csv(io.StringIO(s.decode('utf-8'))).iloc[:,1:]
all_data.replace('nan', np.nan, inplace=True)
all_data



,protocol,date,TVL,Fee,Supply,Staking,Unique Addresses,User Growth,price,amount,...,Active Addresses,Zero Addresses,Number of Transactions,volume,code quality and transparency,biggest drawdown,Correlation,Protocol,Prediction output,Risk Level
0,DYDX,2023-10-19,4.848482e+08,0,259304.857327,0,87.888087,0,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,3.130127e+07,9.0,7.401246e+08,2,DYDX,2,Low
1,Umami,2023-09-08,4.848482e+08,0,259304.857327,0,87.888087,0,4.446382,1.447167e+06,...,191.000000,49.000000,293.000000,3.000337e+05,5.0,9.987676e+08,10,Umami,2,Low
2,Gains Network,2023-10-18,4.848482e+08,0,259304.857327,0,87.888087,0,2.492256,2.863151e+06,...,253.000000,82.000000,367.000000,2.325817e+06,7.0,1.000954e+09,5,Gains Network,2,Low
3,Beefy,2023-10-18,4.848482e+08,0,259304.857327,0,87.888087,0,2.434453,1.743852e+06,...,396.000000,89.000000,545.000000,1.066611e+06,9.0,1.000954e+09,0,Beefy,2,Low
4,LooksRare,2023-11-16,4.848482e+08,0,259304.857327,0,87.888087,0,4.585267,8.044063e+05,...,161.000000,36.000000,200.000000,3.720908e+06,7.0,1.000954e+09,2,LooksRare,2,Low
5,Ribbon Finance,2023-09-04,4.848482e+08,0,259304.857327,0,87.888087,0,1.733587,3.202844e+06,...,369.000000,100.000000,427.000000,2.349799e+05,9.0,8.999332e+08,0,Ribbon Finance,2,Low
6,Redacted Finance,2023-12-01,3.848476e+08,0,39348.987964,0,87.888087,0,4.423593,6.421600e+06,...,433.216495,135.499427,622.235968,2.541630e+05,6.0,8.796980e+08,0,Redacted Finance,0,Low
7,Dopex,2023-06-18,4.848482e+08,0,259304.857327,0,87.888087,0,1.332423,2.556186e+06,...,262.000000,60.000000,405.000000,3.407303e+05,8.0,1.000954e+09,0,Dopex,2,Low
8,GMX,2023-06-14,4.848482e+08,0,259304.857327,0,87.888087,0,1.401703,1.096641e+06,...,249.000000,74.000000,327.000000,2.057191e+07,8.0,9.987676e+08,2,GMX,2,Low
9,Curve,2023-10-18,4.848482e+08,0,259304.857327,0,87.888087,0,4.423593,6.421600e+06,...,433.216495,135.499427,622.235968,8.015657e+07,7.0,1.467449e+08,0,Curve,1,Low


In [ ]:
url = "https://raw.githubusercontent.com/BACCHUS2333/homework2.github.io/main/MergedFinalData3.0.csv"
s=requests.get(url).content
all_data=pd.read_csv(io.StringIO(s.decode('utf-8'))).iloc[:,1:]
all_data.replace('nan', np.nan, inplace=True)


del all_data['Unnamed: 3']
dydx_data = all_data[all_data['protocol'] == 'dydx']

#return the newest from the data
def get_newest(data):
    if data.iloc[-1].isnull().sum() == 0:
        return data.iloc[-1]
    else:
        return data.iloc[-2]
#get_newest(dydx_data)
new_df = pd.DataFrame(columns = all_data.columns)
for protocols in all_data['protocol'].unique():
    new_df.loc[len(new_df)] = list(get_newest(all_data[all_data['protocol'] == protocols]))

#del new_df['date]
#plt.bar(new_df['protocol'], new_df['price'])
rename_mapping = {
    'dydx': 'DYDX',
    'umami': 'Umami',
    'gns': 'Gains Network',
    'bifi': 'Beefy',
    'looks': 'LooksRare',
    'rbn': 'Ribbon Finance',
    'rdt': 'Redacted Finance',
    'dpx': 'Dopex',
    'gmx': 'GMX',
    'crv': 'Curve',
    'snx': 'Synthetix'
}

# Use the `replace` method to rename the values
new_df['protocol'] = new_df['protocol'].replace(rename_mapping)
new_df

,protocol,date,TVL,Fee,Supply,Staking,Unique Addresses,User Growth,price,amount,vol,New Addresses,Active Addresses,Zero Addresses,Number of Transactions,volume,code quality and transparency,biggest drawdown,Correlation
0,DYDX,2023-10-19,4.848482e+08,0,259304.857327,0,87.888087,0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,3.130127e+07,9.0,7.401246e+08,2
1,Umami,2023-09-08,4.848482e+08,0,259304.857327,0,87.888087,0,4.446382,1.447167e+06,1.009808,45.000000,191.000000,49.000000,293.000000,3.000337e+05,5.0,9.987676e+08,10
2,Gains Network,2023-10-18,4.848482e+08,0,259304.857327,0,87.888087,0,2.492256,2.863151e+06,1.393904,55.000000,253.000000,82.000000,367.000000,2.325817e+06,7.0,1.000954e+09,5
3,Beefy,2023-10-18,4.848482e+08,0,259304.857327,0,87.888087,0,2.434453,1.743852e+06,1.302126,131.000000,396.000000,89.000000,545.000000,1.066611e+06,9.0,1.000954e+09,0
4,LooksRare,2023-11-16,4.848482e+08,0,259304.857327,0,87.888087,0,4.585267,8.044063e+05,1.072605,48.000000,161.000000,36.000000,200.000000,3.720908e+06,7.0,1.000954e+09,2
5,Ribbon Finance,2023-09-04,4.848482e+08,0,259304.857327,0,87.888087,0,1.733587,3.202844e+06,0.994619,113.000000,369.000000,100.000000,427.000000,2.349799e+05,9.0,8.999332e+08,0
6,Redacted Finance,2023-12-01,3.848476e+08,0,39348.987964,0,87.888087,0,4.423593,6.421600e+06,1.196620,169.406644,433.216495,135.499427,622.235968,2.541630e+05,6.0,8.796980e+08,0
7,Dopex,2023-06-18,4.848482e+08,0,259304.857327,0,87.888087,0,1.332423,2.556186e+06,0.784137,71.000000,262.000000,60.000000,405.000000,3.407303e+05,8.0,1.000954e+09,0
8,GMX,2023-06-14,4.848482e+08,0,259304.857327,0,87.888087,0,1.401703,1.096641e+06,0.849199,73.000000,249.000000,74.000000,327.000000,2.057191e+07,8.0,9.987676e+08,2
9,Curve,2023-10-18,4.848482e+08,0,259304.857327,0,87.888087,0,4.423593,6.421600e+06,1.196620,169.406644,433.216495,135.499427,622.235968,8.015657e+07,7.0,1.467449e+08,0


In [ ]:
#combine the latest metrics with the final result of transformer
df_for_pre = pd.merge(new_df, final_result, left_on='protocol',right_on='Protocol')
df_for_pre.to_csv('df_for_pre.csv')